# Bond pricing

In this notebook, we show how to compute various fixed-rate bond metrics such as the price, the yield to maturity and the duration. 

## Bond basics

### Price

The price of a bond can be computed as the present value of its future cash flows. Those cash flows include the coupons (if any) and the principal value (or face value) of the bond that is repaid at maturity. Mathematically, the bond price at time $t$ is given by

\begin{equation}
    B_t = P e^{-y(T - t)} + \sum_{i = 1}^N C_i e^{-y(t_i - t)}
\end{equation}

where $B_t$ is the bond price, $P$ is the principal value, $C_i$ is the coupon paid at time $t_i$, $T$ is the maturity time and $y$ is the yield to maturity. Here, we assume that the last coupon is paid at maturity (i.e., $t_N = T$).

Note that $y$ is continuously compounded. Suppose we have a yield of $y_{ann}$ with annual compounding, the corresponding continuously compounded yield is given by

\begin{equation}
    y = \log(1 + y_{ann}).
\end{equation}

This above price is the dirty price of a bond. Between coupon dates, the price quoted by bond dealers, referred to as the clean price, is different from the dirty price. This is because the dirty price includes accrued interest while the clean price does not. The clean price is therefore the dirty price minus accrued interest. Accrued interest is given by

\begin{equation}
    I_{acc} = C_i \frac{t - t_{i-1}}{t_i - t_{i-1}}
\end{equation}

where the numerator is the number of days since the last coupon and the denominator is the number of days from the last coupon date to the next coupon date (i.e., $t_i$ is the time when the next coupon is to be paid and $t_{i-1}$ is the time when the last coupon was paid). Note that there are different conventions to calculate the number of days (e.g., Actual/Actual, 30/360). 

### Yield to maturity

When the bond is traded, its price is readily available. Given the price, we can compute the yield to maturity. This is the value of $y$ that makes the price equation above equal to the market price of the bond. Unless the bond is a zero-coupon bond, $y$ must be found with an optimization algorithm (e.g., Newton). The price of a zero-coupon bond is simply

\begin{equation}
    Z_t = P e^{-y(T - t)}.
\end{equation}

In this case, we can easily derive a closed-form solution for the yield to maturity,

\begin{align}
    Z_t &= P e^{-y(T - t)} \notag \\
    \log Z_t &= \log \left(P e^{-y(T - t)} \right) \notag \\
    \log Z_t &= -y(T - t) + \log P \notag \\
    \log \frac{Z_t}{P} &= -y(T - t) \notag \\
    y &= - \frac{\log \frac{Z_t}{P}}{T - t}.
\end{align}

### Current yield

The current yield is a simple yield measure. It is defined as

\begin{equation}
    y_{curr} = \frac{C}{B_t}.
\end{equation}

### Duration

Let's differentiate the equation for $B_t$ with respect to $y$. We get

\begin{equation}
    \frac{\partial B_t}{\partial y} = -(T - t)P e^{-y(T - t)} - \sum_{i = 1}^N (t_i - t)C_i e^{-y(t_i - t)}.
\end{equation}

The Macaulay duration is given by

\begin{equation}
    D_{mac} = - \frac{1}{B_t} \frac{\partial B_t}{\partial y}.
\end{equation}

The modified duration is given by

\begin{equation}
    D_{mod} = \frac{D_{mac}}{1 + \frac{y_{ann}}{f}}
\end{equation}

where $f$ is the number of coupons paid in a year.

### Convexity

The second order derivative of the bond price with respect to $y$ is given by

\begin{equation}
    \frac{\partial^2B_t}{\partial y^2} = (T - t)^2 P e^{-y(T - t)} + \sum_{i = 1}^N (t_i - t)^2 C_i e^{-y(t_i - t)}.
\end{equation}

The convexity of a bond is given by

\begin{equation}
    \mathcal{C} = \frac{1}{B_t} \frac{\partial^2B_t}{\partial y^2}.
\end{equation}

### Approximating change in price for change in yield

The change in bond price for a given change in yield can be approximated with the modified duration and convexity. Modified duration alone can be used for a small change in yield but for larger changes in yield, it is necessary to use a second order approximation. The percentage change in bond price is given by

\begin{equation}
    \frac{\Delta B_t}{B_t} \approx - D_{mod} \Delta y + \frac{1}{2} \mathcal{C} (\Delta y)^2.
\end{equation}

## Implementation

Let's implement a simple class to compute the bond metrics defined above. The class is designed to price fixed-rate coupon bonds. We assume that coupons are paid at regular intervals over the bond life and that the final coupon is paid on the same date as the principal repayment. The required inputs are the face value of the bond, the coupon rate, the coupon frequency, the settlement date (date at which we price the bond), the maturity date and the day count convention. The available conventions are Actual/Actual (ISMA) and 30/360 (Bond Basis). Beyond that is it necessary to input either the price of the bond (clean or dirty) or the yield to maturity (annual).

It is also possible to price zero-coupon bonds. In this case, set the coupon rate to zero.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy import optimize

class bond_pricer:
    
    def __init__(self, 
                 principal_value, 
                 coupon_rate, 
                 settlement_date, 
                 maturity_date, 
                 coupon_frequency=1,
                 convention='30/360',
                 yield_to_maturity=None, 
                 price_dirty=None,
                 price_clean=None):
        
        self.principal_value = principal_value
        self.coupon_rate = coupon_rate
        self.coupon_frequency = coupon_frequency
        self.coupon_amount = self.principal_value * (self.coupon_rate / self.coupon_frequency)
        self.settlement_date = datetime.strptime(settlement_date, '%d/%m/%Y').date()
        self.maturity_date = datetime.strptime(maturity_date, '%d/%m/%Y').date()
        self.convention = convention
        self.n_months_coupon = int(12 / self.coupon_frequency)  
        
        # Compute cash flow dates
        cf_date = self.maturity_date
        cf_dates = [self.maturity_date]
        while cf_date + relativedelta(months=-self.n_months_coupon) > self.settlement_date:
            cf_date += relativedelta(months=-self.n_months_coupon)
            cf_dates.append(cf_date)        
        self.cash_flow_dates = cf_dates[::-1]
        
        # Compute accrued interest
        self.next_coupon_date = self.cash_flow_dates[0]
        self.prev_coupon_date = self.next_coupon_date + relativedelta(months=-self.n_months_coupon)
        if self.convention == 'Actual/Actual':
            self.n_days_accrued = (self.settlement_date - self.prev_coupon_date).days    
            self.n_days_curr_coupon = (self.next_coupon_date - self.prev_coupon_date).days
            self.accrual_period = self.n_days_accrued / self.n_days_curr_coupon
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        elif self.convention == '30/360':
            d1 = min(30, self.prev_coupon_date.day)
            if d1 == 30:
                d2 = min(d1, self.settlement_date.day)
            else:
                d2 = self.settlement_date.day
            self.n_days_accrued = 360 * (self.settlement_date.year - self.prev_coupon_date.year) + 30 * (self.settlement_date.month - self.prev_coupon_date.month) + d2 - d1
            self.n_days_curr_coupon = 360 / self.coupon_frequency
            self.accrual_period = self.n_days_accrued / self.n_days_curr_coupon
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        
        # Compute cash flow amounts and time to cash flows and maturity
        first_tau = (self.n_days_curr_coupon - self.n_days_accrued) / self.n_days_curr_coupon        
        cf_amounts = []
        taus = []
        for i in range(len(self.cash_flow_dates)):
            cf_amounts.append(self.coupon_amount)
            taus.append((first_tau + i) / self.coupon_frequency)        
        cf_amounts[-1] += self.principal_value
        self.cash_flows = np.array(cf_amounts)
        self.taus = np.array(taus)
        self.time_to_maturity = self.taus[-1]
        
        # Compute clean and dirty prices
        if price_dirty:
            self.price_dirty = price_dirty
            self.price_clean = self.price_dirty - self.accrued_interest
        elif price_clean:
            self.price_clean = price_clean
            self.price_dirty = self.price_clean + self.accrued_interest
        elif yield_to_maturity:
            self.yield_to_maturity = yield_to_maturity
            self.ytm_adjusted = self.coupon_frequency*np.log(1 + self.yield_to_maturity / self.coupon_frequency)
            self.discount_factors = np.exp(-self.ytm_adjusted * self.taus)
            self.price_dirty = np.sum(self.cash_flows * self.discount_factors)
            self.price_clean = self.price_dirty - self.accrued_interest
        
        # Compute current yield
        self.current_yield = (self.coupon_rate * self.principal_value) / self.price_dirty
        
        # Compute yield to maturity
        if yield_to_maturity:
            self.yield_to_maturity = yield_to_maturity
            self.ytm_adjusted = self.coupon_frequency * np.log(1 + self.yield_to_maturity / self.coupon_frequency)
        else:
            def get_bond_price(yield_to_maturity, cash_flows, taus):
                return np.sum(cash_flows * np.exp(-yield_to_maturity * taus))
            
            get_ytm = lambda ytm: get_bond_price(ytm, self.cash_flows, self.taus) - self.price_dirty
            
            self.ytm_adjusted = optimize.newton(get_ytm, 0.04)
            self.yield_to_maturity = (np.exp(self.ytm_adjusted / self.coupon_frequency) - 1) * self.coupon_frequency
            self.discount_factors = np.exp(-self.ytm_adjusted * self.taus)
       
        # Compute duration
        self.first_derivative = np.sum(-self.taus * self.cash_flows * self.discount_factors)
        self.duration_macaulay = -1/self.price_dirty * self.first_derivative
        self.duration_modified = self.duration_macaulay / (1 + self.yield_to_maturity / self.coupon_frequency)                                                        
        
        # Compute convexity
        self.second_derivative = np.sum(self.taus**2 * self.cash_flows * self.discount_factors)
        self.convexity = 1/self.price_dirty * self.second_derivative
        
        # Generate summary table
        index = ['Dirty price','Clean price', 'Accrued interest', 'Face value', 'Coupon rate', 'Coupon frequency', 'Yield to maturity', 
                 'Time to maturity', 'Macaulay duration', 'Modified duration', 'Convexity', 'Settlement date', 'Maturity date', 'Convention']
        data = list(np.round([self.price_dirty, self.price_clean, self.accrued_interest, self.principal_value, self.coupon_rate, self.coupon_frequency, 
                self.yield_to_maturity, self.time_to_maturity, self.duration_macaulay, self.duration_modified, self.convexity],2))
        data += [self.settlement_date, self.maturity_date, self.convention]
        self.summary = pd.DataFrame(data, index, columns=[''])
        
    # Approximate new price after delta_ytm change in yield to maturity
    def approx_new_price(self, delta_ytm):
        return self.price_dirty * (1 + (-self.duration_modified * delta_ytm + 0.5 * self.convexity * delta_ytm**2))
    
    # Summary of the bond metrics
    def print_summary(self):
        print(self.summary)

Let's define some parameters and price a bond.

In [2]:
# Parameters
principal_value = 100
coupon_rate = 0.0684
coupon_frequency = 12
settlement_date = '24/01/2019'
maturity_date = '19/04/2027'
yield_to_maturity = 0.05
convention='30/360'

# Instantiate bond
bond = bond_pricer(principal_value=principal_value, coupon_rate=coupon_rate, coupon_frequency=coupon_frequency, 
                settlement_date=settlement_date, maturity_date=maturity_date, 
                yield_to_maturity=yield_to_maturity, convention=convention)

# Print bond metrics
bond.print_summary()

                             
Dirty price             112.5
Clean price             112.4
Accrued interest          0.1
Face value              100.0
Coupon rate              0.07
Coupon frequency         12.0
Yield to maturity        0.05
Time to maturity         8.24
Macaulay duration        6.44
Modified duration        6.42
Convexity               48.44
Settlement date    2019-01-24
Maturity date      2027-04-19
Convention             30/360


We can also check that we get a consistent yield to maturity when plugging the bond price. 

In [3]:
# Parameters
principal_value = 100
coupon_rate = 0.0684
coupon_frequency = 12
settlement_date = '24/01/2019'
maturity_date = '19/04/2027'
price_dirty = bond.price_dirty
convention='30/360'

# Instantiate bond
bond2 = bond_pricer(principal_value=principal_value, coupon_rate=coupon_rate, coupon_frequency=coupon_frequency, 
                settlement_date=settlement_date, maturity_date=maturity_date, 
                yield_to_maturity=yield_to_maturity, convention=convention)

# Print bond metrics
bond2.print_summary()

                             
Dirty price             112.5
Clean price             112.4
Accrued interest          0.1
Face value              100.0
Coupon rate              0.07
Coupon frequency         12.0
Yield to maturity        0.05
Time to maturity         8.24
Macaulay duration        6.44
Modified duration        6.42
Convexity               48.44
Settlement date    2019-01-24
Maturity date      2027-04-19
Convention             30/360


Let's compare our metrics to those obtained with Quantlib.

In [4]:
import QuantLib as ql

# Parameters
coupon_period = ql.Monthly
#convention = ql.ActualActual(ql.ActualActual.ISMA)
convention = ql.Thirty360()
faceValue = 100.
couponType = coupon_period
issueDate = ql.Date(19, 4, 2017)
EvalDate = ql.Date(24, 1, 2019)
Maturity = ql.Date(19, 4, 2027)
quotedYield = 5
couponRate = 6.84
calendar = ql.NullCalendar()
settlementDays = 0
businessConventions = ql.Following
Datesgeneration = ql.DateGeneration.Forward
period= ql.Period(coupon_period)
monthEnd = False

# Instantiate bond
ql.Settings.instance().evaluationDate = EvalDate
schedule = ql.Schedule(issueDate, Maturity, period, calendar, businessConventions, businessConventions, Datesgeneration, monthEnd)
ql_bond = ql.FixedRateBond(settlementDays, faceValue, schedule, [couponRate/100.], convention)

# Print metrics
print("QuantLib dirty price:       ", ql_bond.dirtyPrice(quotedYield/100., convention, ql.Compounded, coupon_period))
print("Our dirty price:            ", bond.price_dirty)
print("QuantLib clean price:       ", ql_bond.cleanPrice(quotedYield/100., convention, ql.Compounded, coupon_period))
print("Our clean price:            ", bond.price_clean)
print("QuantLib Macaulay duration: ", ql.BondFunctions.duration(ql_bond,quotedYield/100,convention, ql.Compounded, coupon_period, ql.Duration.Macaulay))
print("Our Macaulay duration:      ", bond.duration_macaulay)
print("QuantLib accrued interest:  ", ql_bond.accruedAmount())
print("Our accrued interest:       ", bond.accrued_interest)

QuantLib dirty price:        112.49569655265081
Our dirty price:             112.49569655265047
QuantLib clean price:        112.40069655265081
Our clean price:             112.40069655265047
QuantLib Macaulay duration:  6.443516786214288
Our Macaulay duration:       6.443516786214299
QuantLib accrued interest:   0.09500000000000064
Our accrued interest:        0.095
